In [1]:
from data_handling import *
from networks import *
from IPython.display import display
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from torch.utils.data import DataLoader
from torch import nn, optim

import random
random.seed(1)

from enum import Enum

class model_type(Enum):
    U_Net = 1,
    DeepLabV3 = 2,
    L_U_Net = 3,

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
patches_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)
def load_model(model:model_type,num_classes=4):
    if model == model_type.U_Net:
        return UNet(num_classes)
    elif model == model_type.DeepLabV3:
        return deeplabv3_resnet50(num_classes=num_classes)
    elif model == model_type.L_U_Net:
        return L_U_Net(num_classes,16)
class EarlyStopping:
    def __init__(self, patience=20, delta=0,file_name:str="checkpoint.pth"):
        """
        Args:
            patience (int): How many epochs should we wait after last time validation loss improved.
                            Default: 20
            delta (float): Minimum change in the loss to qualify as an improvement.
                           Default: 0
        """
        self.patience = patience
        self.counter = 0
        self.val_loss_min = float('inf')
        self.delta = delta
        self.early_stop = False
        self.name = file_name

    def __call__(self, val_loss, model,verbose=False):

        if (val_loss + self.delta < self.val_loss_min):
            self.save_checkpoint(val_loss, model,verbose)
            self.val_loss_min = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

    def save_checkpoint(self, val_loss, model,verbose):
        if(verbose):
            print(f'Validation loss decreased ({self.val_loss_min:.6f} -> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.name)
        self.val_loss_min = val_loss

Using device: cuda


In [3]:
# Parameters
batch_size = 4
num_epochs = 200
learning_rate = 1e-4
decay = 1e-5
num_trials = 10
num_classes = 4
losses = [[[] for _ in range(num_trials)],[[] for _ in range(num_trials)]] # training , validation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for i in range(num_trials):

    #Change the enum to U_Net to train a U-net model.
    model = load_model(model_type.L_U_Net,num_classes=num_classes)
    model.to(device)

    # Get data
    train_subset,val_subset,_ = generate_set(patch_size=224,resize=(1120, 1344),filepath="D:/Skola/Avancerad AI/",indices=i*2)

    # Done to created weights for the loss function, it might be better to do this on patches to include the true distribution based on dynamically added crops
    class_counts = np.zeros(num_classes)
    for page in train_subset: #Sum up the number of pixels for each class for each page
        for class_id in range(num_classes):
            class_counts[class_id] += np.sum(page.gt == class_id)

    total_pixels = np.sum(class_counts) #Count the total of pixels
    frequencies = class_counts / total_pixels
    weights = torch.tensor(np.sqrt(1.0 / frequencies).astype('float32')) # calculate weights
    weights
    weights = weights.to(device)
    # Loss, optimizer, stopper
    criterion = nn.CrossEntropyLoss(weight=weights)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=decay)
    earlystopper= EarlyStopping(20,0, f'./model-small-fold-{i}.pth')


    #Prepare data
    train_dataset = PatchesDataset(train_subset,patches_transforms)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = PatchesDataset(val_subset,patches_transforms)
    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

    # Training loop
    for epoch in range(num_epochs):

        train_dataset.random_patch_generator(10)
        val_dataset.random_patch_generator(10)
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:

            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)['out']
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * labels.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        losses[0].append(train_loss)

        #Validation loop
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)['out']
                loss = criterion(outputs,labels)
                running_loss += loss.item() * labels.size(0)
        val_loss = running_loss/len(val_loader.dataset)
        losses[1][i].append(val_loss)
        if epoch > 50:
            earlystopper(val_loss=val_loss,model=model,verbose=False)
        if earlystopper.early_stop == True:
                break
        print(f'Epoch [{epoch+1}/{num_epochs}], Train loss: {loss.item():.4f}, Validation loss {val_loss:.4f}')

    print(f'Training finished! after {epoch} epochs, Model saved to ./model-fold-{i}.pth')

KeyboardInterrupt: 

In [5]:

def train_model(model_arch:model_type,fold_path,batch_size = 4,
num_epochs = 200,
learning_rate = 1e-4,
decay = 1e-5,
num_trials = 10,
num_classes = 4,
num_pages = 2,
patch_size=224,resize=(1120,1344)):
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)

    losses = [[[] for _ in range(num_trials)],[[] for _ in range(num_trials)]] # training , validation
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for i in range(num_trials):

        #Change the enum to U_Net to train a U-net model.
        model = load_model(model_arch,num_classes=num_classes)
        model.to(device)

        # Get data
        train_subset,val_subset,_ = generate_set(patch_size=patch_size,resize=resize,filepath="D:/Skola/Avancerad AI/",indices=i*num_pages,num_pages=num_pages)

        # Done to created weights for the loss function, it might be better to do this on patches to include the true distribution based on dynamically added crops
        class_counts = np.zeros(num_classes)
        for page in train_subset: #Sum up the number of pixels for each class for each page
            for class_id in range(num_classes):
                class_counts[class_id] += np.sum(page.gt == class_id)

        total_pixels = np.sum(class_counts) #Count the total of pixels
        frequencies = class_counts / total_pixels
        weights = torch.tensor(np.sqrt(1.0 / frequencies).astype('float32')) # calculate weights
        weights
        weights = weights.to(device)
        # Loss, optimizer, stopper
        criterion = nn.CrossEntropyLoss(weight=weights)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=decay)
        earlystopper= EarlyStopping(20,0, f'./{fold_path}/model-small-fold-{i}.pth')


        #Prepare data
        train_dataset = PatchesDataset(train_subset,patches_transforms)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        val_dataset = PatchesDataset(val_subset,patches_transforms)
        val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

        # Training loop
        for epoch in range(num_epochs):

            train_dataset.random_patch_generator(10)
            val_dataset.random_patch_generator(10)
            model.train()
            running_loss = 0.0
            for images, labels in train_loader:

                images, labels = images.to(device), labels.to(device)

                # Forward pass
                outputs = model(images)['out']
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * labels.size(0)

            train_loss = running_loss / len(train_loader.dataset)
            losses[0][i].append(train_loss)

            #Validation loop
            model.eval()
            running_loss = 0.0
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)['out']
                    loss = criterion(outputs,labels)
                    running_loss += loss.item() * labels.size(0)
            val_loss = running_loss/len(val_loader.dataset)
            losses[1][i].append(val_loss)
            if epoch > 50:
                earlystopper(val_loss=val_loss,model=model,verbose=False)
            if earlystopper.early_stop == True:
                    break
            print(f'Epoch [{epoch+1}/{num_epochs}], Train loss: {loss.item():.4f}, Validation loss {val_loss:.4f}')

        print(f'Training finished! after {epoch} epochs, Model saved to ./model-fold-{i}.pth')
    return losses

losses = train_model(model_type.L_U_Net,'L-unet-folds')
losses2 = train_model(model_type.U_Net,'unet-folds')


Epoch [1/200], Train loss: 0.9604, Validation loss 1.0720
Epoch [2/200], Train loss: 0.9667, Validation loss 1.0302
Epoch [3/200], Train loss: 1.0803, Validation loss 0.9731
Epoch [4/200], Train loss: 0.9255, Validation loss 0.8948
Epoch [5/200], Train loss: 0.8964, Validation loss 0.8515
Epoch [6/200], Train loss: 0.7776, Validation loss 0.8097
Epoch [7/200], Train loss: 0.6740, Validation loss 0.7065
Epoch [8/200], Train loss: 0.7098, Validation loss 0.6804
Epoch [9/200], Train loss: 0.8940, Validation loss 0.6806
Epoch [10/200], Train loss: 0.6275, Validation loss 0.5988
Epoch [11/200], Train loss: 0.8555, Validation loss 0.6652
Epoch [12/200], Train loss: 0.8095, Validation loss 0.5902
Epoch [13/200], Train loss: 0.5464, Validation loss 0.5674
Epoch [14/200], Train loss: 0.4631, Validation loss 0.6138
Epoch [15/200], Train loss: 0.5176, Validation loss 0.5461
Epoch [16/200], Train loss: 0.7484, Validation loss 0.4826
Epoch [17/200], Train loss: 0.4501, Validation loss 0.4877
Epoch 

In [ ]:
folder_path = "hello"

else:
    print(f"Folder '{folder_path}' already exists.")

Folder 'hello' already exists.
